In [1]:
# this was a main branch line but we added some more text after creating a new branch branch_1 while staying in branch branch_1
import pymysql
import pandas as pd
import re
from textblob import TextBlob as tb
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from multiprocessing import cpu_count
from multiprocessing import Pool
from sklearn.model_selection import train_test_split
import numpy as np
from keras.utils import np_utils
import gensim
from gensim.models.word2vec import Word2Vec 
from sklearn.preprocessing import scale
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras import optimizers
from keras.layers import Activation
from scipy.special import softmax
from keras.models import model_from_json
import pickle

def amazon_lable(x):
    a=-1
    if(x=='POSITIVE'):
        a = 1
    if(x=='NEGATIVE'):
        a = 0
    if(x=='NEUTRAL'):
        a = 2
    return a

def extract_words(temp):
    #temp = re.sub(r'[^\w\s]','',temp) #punctuation
    temp = re.sub(r'[^a-zA-Z0-9\s]', ' ', temp)
    temp = re.sub(r'\d+','',temp) #remove numbers
    temp = temp.lower()
    temp = tb(temp)
    lista = [word for word in temp.words if word not in stopwords.words('english')]
    listb = lista
    listb = [lemmatizer.lemmatize(y) for y in lista]
    #listb = [stemmer.stem(y) for y in listb]
    listb = [word for word in listb if len(word)>2]
    #listb = list(set(listb))
    return(listb)

#not necessary for elmo algo
# def buildWordVector(tokens, size):
#     vec = np.zeros(size).reshape((1, size))
#     count = 0.
#     for word in tokens:
#         try:
#             vec += google_w2vec[word].reshape((1, size))
#             count += 1.
#         except KeyError: # handling the case where the token is not
#                          # in the corpus. useful for testing.  * tfidf[word]
#             continue
#     if count != 0:
#         vec /= count
#     return vec

def get_lable(b):
    a = list(b)
    i = a.index(max(a))
    if(i==0):
        lable = 0
    if(i==1):
        lable = 1
    if(i==2):
        lable = 2
    return lable

def compare(x1,x2):
    if(x1==x2):
        c = 0
    else:
        c = 1
    return c

def get_precision(x):
    df = x
    print("Precision for negatives:" +str(df.iloc[0][0]/sum(df[0])))
    print("Precision for positives:" + str(df.iloc[1][1]/sum(df[1])))
    print("Precision for neutrals:" + str(df.iloc[2][2]/sum(df[2])))

connection = pymysql.connect(host='ec2-174-129-19-167.compute-1.amazonaws.com',
    							 user='umtdev',
    							 password='GX^2M$4%781!ra1t-e~',
    							 db='unmetric',
    							 charset='utf8mb4',
    							 cursorclass=pymysql.cursors.DictCursor,autocommit = True)
cursor = connection.cursor()

query = str("select a.fbcomment_id,a.text,b.sentiment from fb_comment_text a join\
            fb_sentiment_comments_comprehend b on b.fbpagepostcomments_id = a.fbcomment_id;")
request = cursor.execute(query)
comments_amazon = pd.DataFrame(cursor.fetchall())
cursor.close()
connection.close()

comments_amazon['senti_lable'] = comments_amazon['sentiment'].apply(lambda x : amazon_lable(x))

pool = Pool(processes=cpu_count()-1)   
print("Starting Pooling...")
results =  pool.map(extract_words, comments_amazon['text'])
print("Pooling Finished...")
pool.close()
pool.join()
comments_amazon['tokens'] = results
print(comments_amazon.head())

data = comments_amazon[{'tokens','senti_lable','fbcomment_id'}]
data['clean_comment'] = data['tokens'].apply(lambda x : ' '.join(x))

ids_train = pd.read_csv("amazon_train_ids.csv")

ids_train = np.array(ids_train['0'])

data_train = data[data.fbcomment_id.isin(ids_train)]

#x_train = np.array(data_train.tokens)

y_train = np.array(data_train.senti_lable)
y_train = np_utils.to_categorical(y_train)

ids_test = pd.read_csv("amazon_test_ids.csv")

ids_test = np.array(ids_test['0'])

data_test = data[data.fbcomment_id.isin(ids_test)]

#x_test = np.array(data_test.tokens)

y_test = np.array(data_test.senti_lable)
y_test = np_utils.to_categorical(y_test)

Using TensorFlow backend.


Starting Pooling...
Pooling Finished...
   fbcomment_id sentiment                                               text  \
0    9065432149   NEUTRAL          We would upgrade to the A380 anyway! Haha   
1    8841832192  NEGATIVE  "They understand this and have no problem reso...   
2    8841832194  NEGATIVE  #topthreenotsocustomerservicemoments After bum...   
3    8841832196   NEUTRAL          Hey Laurie, can you send both please? ^EH   
4    8967296634  NEGATIVE  I am so incredibly disappointed with my servic...   

   senti_lable                                             tokens  
0            2                     [would, upgrade, anyway, haha]  
1            0  [understand, problem, resolving, baggage, issu...  
2            0  [topthreenotsocustomerservicemoments, bumping,...  
3            2                        [hey, laurie, send, please]  
4            0  [incredibly, disappointed, service, united, fa...  


/root/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)

In [7]:
def elmo_vectors(x):
    embeddings = elmo(x.tolist(), signature="default", as_dict=True)["default"]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))

In [4]:
list_test = [data_test[i:i+1000] for i in range(0,data_test.shape[0],1000)]

In [5]:
list_train = [data_train[i:i+1000] for i in range(0,data_train.shape[0],1000)]

In [8]:
elmo_train = [None] * len(list_train)

In [ ]:
for i in range(0,90):
    print(i)
    x = list_train[i]
    elmo_train[i] = elmo_vectors(x['clean_comment'])
    if(i%20==0 or i==89):
        pickle_out = open("elmo_train_comprehend_default1.pickle","wb")
        pickle.dump(elmo_train, pickle_out)
        pickle_out.close()

In [25]:
pickle_out = open("elmo_train_comprehend.pickle","wb")
pickle.dump(elmo_train, pickle_out)
pickle_out.close()

In [4]:
elmo_test = [None] * len(list_test)

In [ ]:
for i in range(4,30):
    print(i)
    x = list_test[i]
    elmo_test[i] = elmo_vectors(x['clean_comment'])
    if(i%20==0 or i==29):
        pickle_out = open("elmo_test_comprehend.pickle","wb")
        pickle.dump(elmo_test, pickle_out)
        pickle_out.close()

In [2]:
#load elmo training data
pickle_in = open("elmo_train_comprehend_all.pickle", "rb")
elmo_train_total = pickle.load(pickle_in)
elmo_train_total = np.concatenate(elmo_train_total, axis = 0)

In [7]:
#load elmo test data
pickle_in = open("elmo_test_comprehend.pickle", "rb")
elmo_test_comprehend = pickle.load(pickle_in)
elmo_test_comprehend_all = [None]*len(elmo_test_comprehend)
elmo_test_comprehend_all[0:30] = elmo_test_comprehend[0:30]
pickle_in = open("elmo_test_comprehend2.pickle", "rb")
elmo_test_comprehend2 = pickle.load(pickle_in)
elmo_test_comprehend_all[30:len(elmo_test_comprehend2)] = elmo_test_comprehend2[30:len(elmo_test_comprehend2)]
elmo_test_total = np.concatenate(elmo_test_comprehend_all, axis = 0)
elmo_test_scaled = scale(elmo_test_total)

/root/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/root/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/data.py:193: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [ ]:
pickle_out = open("elmo_test_comprehend_all.pickle","wb")
pickle.dump(elmo_test_total, pickle_out)
pickle_out.close()

In [3]:
model = Sequential()
model.add(Dense(32, activation='elu', input_dim=1024))

model.add(Dropout(0.1))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.1))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.1))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.1))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.1))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.1))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.1))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.1))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.1))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.1))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.1))
model.add(Dense(32))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.1))

model.add(Dense(3,activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']) 

In [4]:
elmo_train_scaled = scale(elmo_train_total)

/root/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/root/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/data.py:193: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [5]:
model.fit(elmo_train_scaled, y_train, epochs=50, batch_size=32, verbose=2,validation_split=0.1)

Train on 240447 samples, validate on 26717 samples
Epoch 1/50
 - 37s - loss: 0.3704 - acc: 0.8419 - val_loss: 0.3837 - val_acc: 0.8417
Epoch 2/50
 - 36s - loss: 0.3421 - acc: 0.8544 - val_loss: 0.3520 - val_acc: 0.8466
Epoch 3/50
 - 35s - loss: 0.3342 - acc: 0.8581 - val_loss: 0.3420 - val_acc: 0.8492
Epoch 4/50
 - 36s - loss: 0.3292 - acc: 0.8603 - val_loss: 0.3413 - val_acc: 0.8508
Epoch 5/50
 - 35s - loss: 0.3248 - acc: 0.8616 - val_loss: 0.3440 - val_acc: 0.8474
Epoch 6/50
 - 36s - loss: 0.3212 - acc: 0.8636 - val_loss: 0.3456 - val_acc: 0.8495
Epoch 7/50
 - 36s - loss: 0.3184 - acc: 0.8648 - val_loss: 0.3445 - val_acc: 0.8441
Epoch 8/50
 - 36s - loss: 0.3166 - acc: 0.8652 - val_loss: 0.3449 - val_acc: 0.8494
Epoch 9/50
 - 35s - loss: 0.3145 - acc: 0.8665 - val_loss: 0.3337 - val_acc: 0.8544
Epoch 10/50
 - 36s - loss: 0.3144 - acc: 0.8664 - val_loss: 0.3348 - val_acc: 0.8537
Epoch 11/50
 - 36s - loss: 0.3114 - acc: 0.8679 - val_loss: 0.3354 - val_acc: 0.8531
Epoch 12/50
 - 37s - lo

In [6]:
dum_pred = model.predict(elmo_train_scaled,batch_size=32,verbose=1)
dum_pred = softmax(dum_pred,axis=1)
train_prediction = list(map(get_lable,dum_pred))
y_train_lable = list(map(get_lable,y_train))
difference = list(map(compare,train_prediction,y_train_lable))

print("Training Accuracy : "+str((len(difference)-sum(difference))/(len(difference))))

267164/267164 [==============================] - 10s 38us/step
Training Accuracy : 0.8260544085280951


In [8]:
dum_pred = model.predict(elmo_test_scaled,batch_size=32,verbose=1)
dum_pred = softmax(dum_pred,axis=1)
test_prediction = list(map(get_lable,dum_pred))
y_test_lable = list(map(get_lable,y_test))
difference = list(map(compare,test_prediction,y_test_lable))

print("Test Accuracy : "+str((len(difference)-sum(difference))/(len(difference))))

66791/66791 [==============================] - 2s 37us/step
Test Accuracy : 0.7984608704765612


In [9]:
y_actu = pd.Series(y_test_lable, name='Actual')
y_pred = pd.Series(test_prediction, name='Predicted')
df = pd.crosstab(y_actu, y_pred)
get_precision(df)
print("Recall for negatives :" + str(df.iloc[0][0]/sum(df.iloc[0])))
print("Recall for positives :" + str(df.iloc[1][1]/sum(df.iloc[1])))
print("Recall for neutrals :" + str(df.iloc[2][2]/sum(df.iloc[2])))

Precision for negatives:0.7803052396228249
Precision for positives:0.8019934963947406
Precision for neutrals:0.8085497801752362
Recall for negatives :0.8329355608591885
Recall for positives :0.7116868452418292
Recall for neutrals :0.8212249809982265


In [10]:
apple = pd.read_csv("apple.csv")
apple = apple[apple.sentiment!="not_relevant"]
apple['tokens'] = apple['text'].apply(lambda x : extract_words(x))
apple['clean_comment'] = apple['tokens'].apply(lambda x : ' '.join(x))
print(apple.head())

def apple_lable(x):
    if(x=='1'):
        a = 0
    if(x=='3'):
        a = 2
    if(x=='5'):
        a = 1
    return a

y_apple= np.array(apple.sentiment)

y_apple = list(map(apple_lable,y_apple))

y_apple = np_utils.to_categorical(y_apple)

  sentiment  confidence_score  \
0         3            0.8129   
1         3            1.0000   
2         5            0.8468   
3         3            0.7997   
4         1            1.0000   

                                                text  \
0  RT @JPDesloges: Why AAPL Stock Had a Mini-Flas...   
1  My cat only chews @apple cords. Such an #Apple...   
2  Top 3 all @Apple #tablets. Damn right! http://...   
3  Apple Inc. Flash Crash: What You Need to Know ...   
4  WTF MY BATTERY WAS 31% ONE SECOND AGO AND NOW ...   

                                              tokens  \
0  [jpdesloges, aapl, stock, mini, flash, crash, ...   
1                [cat, chew, apple, cord, applesnob]   
2  [top, apple, tablet, damn, right, http, rjignj...   
3  [apple, inc, flash, crash, need, know, http, y...   
4       [wtf, battery, one, second, ago, wtf, apple]   

                                       clean_comment  
0  jpdesloges aapl stock mini flash crash today a...  
1                

In [16]:
# n_dim = 300
# apple_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in map(lambda x: x, x_apple)])
# apple_vecs_w2v = scale(apple_vecs_w2v)
list_apple = [apple[i:i+1000] for i in range(0,apple.shape[0],1000)]

elmo_apple = [None]*len(list_apple)

for i in range(0,len(elmo_apple)):
    print(i)
    x = list_apple[i]
    elmo_apple[i] = elmo_vectors(x['clean_comment'])
    if(i==len(elmo_apple)):
        pickle_out = open("elmo_apple.pickle","wb")
        pickle.dump(elmo_apple, pickle_out)
        pickle_out.close()

In [17]:
elmo_apple = np.concatenate(elmo_apple,axis = 0)

In [19]:
elmo_apple_scaled = scale(elmo_apple)

/root/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/root/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/data.py:193: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [21]:
dum_pred = model.predict(elmo_apple_scaled,batch_size=32,verbose=1)
dum_pred = softmax(dum_pred,axis=1)
apple_prediction = list(map(get_lable,dum_pred))
y_apple_lable = list(map(get_lable,y_apple))
difference = list(map(compare,apple_prediction,y_apple_lable))

print("Apple Accuracy : "+str((len(difference)-sum(difference))/(len(difference))))

2359/2359 [==============================] - 0s 41us/step
Apple Accuracy : 0.7049597286986011


In [ ]:
airline = pd.read_csv("airline.csv")
airline['tokens'] = airline['text'].apply(lambda x : extract_words(x))
airline['clean_comment'] = airline['tokens'].apply(lambda x : ' '.join(x))

print(airline.head())

def airline_lable(x):
    if(x=='negative'):
        a = 0
    if(x=='neutral'):
        a = 2
    if(x=='positive'):
        a = 1
    return a

y_airline= np.array(airline.airline_sentiment)
y_airline = list(map(airline_lable,y_airline))
y_airline = np_utils.to_categorical(y_airline)

list_airline = [airline[i:i+1000] for i in range(0,airline.shape[0],1000)]
elmo_airline = [None]*len(list_airline)

In [27]:
for i in range(0,len(elmo_airline)):
    print(i)
    x = list_airline[i]
    elmo_airline[i] = elmo_vectors(x['clean_comment'])
    if(i==(len(elmo_airline)-1)):
        pickle_out = open("elmo_airline.pickle","wb")
        pickle.dump(elmo_airline, pickle_out)
        pickle_out.close()
elmo_airline = np.concatenate(elmo_airline,axis = 0)

0
1
2
3
4
5
6
7
8
9
10


In [32]:
elmo_airline_scaled = scale(elmo_airline)
dum_pred = model.predict(elmo_airline_scaled,batch_size=32,verbose=1)
dum_pred = softmax(dum_pred,axis=1)
airline_prediction = list(map(get_lable,dum_pred))
y_airline_lable = list(map(get_lable,y_airline))
difference = list(map(compare,airline_prediction,y_airline_lable))

print("Airline Accuracy : "+str((len(difference)-sum(difference))/(len(difference))))

 2816/10768 [======>.......................] - ETA: 0s

/root/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/root/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/data.py:193: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


10768/10768 [==============================] - 0s 36us/step
Airline Accuracy : 0.6053120356612184


In [33]:
#self drive cars dataset
selfdrive = pd.read_csv("selfdrive.csv")
selfdrive = selfdrive[~selfdrive.sentiment.isin(['not_relevant','2','4'])]
selfdrive['tokens'] = selfdrive['text'].apply(lambda x : extract_words(x))
selfdrive['clean_comment'] = selfdrive['tokens'].apply(lambda x : ' '.join(x))

print(selfdrive.head())

def selfdrive_lable(x):
    if(x=='1'):
        a = 0
    if(x=='3'):
        a = 2
    if(x=='5'):
        a = 1
    return a

y_selfdrive= np.array(selfdrive.sentiment)
y_selfdrive = list(map(selfdrive_lable,y_selfdrive))
y_selfdrive = np_utils.to_categorical(y_selfdrive)

list_selfdrive = [selfdrive[i:i+1000] for i in range(0,selfdrive.shape[0],1000)]
elmo_selfdrive = [None]*len(list_selfdrive)

  sentiment  confidence_score  \
0         5            0.7579   
1         5            0.8775   
3         3            1.0000   
4         3            1.0000   
5         5            0.7187   

                                                text  \
0  Two places I'd invest all my money if I could:...   
1  Awesome! Google driverless cars will help the ...   
3  Just saw Google self-driving car on I-34. It w...   
4  Will driverless cars eventually replace taxi d...   
5  Autonomous vehicles could reduce traffic fatal...   

                                              tokens  \
0  [two, place, invest, money, could, printing, s...   
1  [awesome, google, driverless, car, help, blind...   
3  [saw, google, self, driving, car, painted, gre...   
4  [driverless, car, eventually, replace, taxi, d...   
5  [autonomous, vehicle, could, reduce, traffic, ...   

                                       clean_comment  
0  two place invest money could printing self dri...  
1  awesome google

In [35]:
for i in range(0,len(elmo_selfdrive)):
    print(i)
    x = list_selfdrive[i]
    elmo_selfdrive[i] = elmo_vectors(x['clean_comment'])
    if(i==(len(elmo_selfdrive)-1)):
        pickle_out = open("elmo_selfdrive.pickle","wb")
        pickle.dump(elmo_selfdrive, pickle_out)
        pickle_out.close()
elmo_selfdrive = np.concatenate(elmo_selfdrive,axis = 0)

0
1
2
3


In [47]:
#elmo_selfdrive_scaled= scale(elmo_selfdrive)
dum_pred = model.predict(elmo_selfdrive_scaled,batch_size=32,verbose=1)
dum_pred = softmax(dum_pred,axis=1)
selfdrive_prediction = list(map(get_lable,dum_pred))
y_selfdrive_lable = list(map(get_lable,y_selfdrive))
difference = list(map(compare,selfdrive_prediction,y_selfdrive_lable))

print("selfdrive Accuracy : "+str((len(difference)-sum(difference))/(len(difference))))

3727/3727 [==============================] - 0s 37us/step
selfdrive Accuracy : 0.06412664341293266


In [53]:
preprocessing.normalize([[-1,0,1,2]])

array([[-0.40824829,  0.        ,  0.40824829,  0.81649658]])

In [54]:
surfnew = pd.read_csv("FB_COMMENTS__Surf_Excel_India_Mar_11___10_59_PM.csv",encoding='ISO-8859-1')

surfnew['tokens'] = surfnew['Comment Text'].apply(lambda x : extract_words(x))
surfnew['clean_comment'] = surfnew['tokens'].apply(lambda x : ' '.join(x))

In [56]:
def remove_empty_tokens(x):
    if(len(x) == 0 ):
        return(float('NaN'))
    else:
        return(100)

In [148]:
surfnew['check_tokens'] = surfnew['tokens'].apply(lambda x : remove_empty_tokens(x) )

surfnew =surfnew[pd.notnull(surfnew['check_tokens'])]

list_surf = [surfnew[i:i+1000] for i in range(0,surfnew.shape[0],1000)]

elmo_surf = [None]*len(list_surf)

In [19]:
for i in range(41,len(elmo_surf)):
    print(i)
    x = list_surf[i]
    elmo_surf[i] = elmo_vectors(x['clean_comment'])
    if(i==(len(elmo_surf)-1)):
        pickle_out = open("elmo_surf.pickle","wb")
        pickle.dump(elmo_surf, pickle_out)
        pickle_out.close()

41


In [34]:
pickle_in = open("elmo_surf.pickle", "rb")
elmo_surf = pickle.load(pickle_in)

In [33]:
# pickle_out = open("elmo_surf.pickle","wb")
# pickle.dump(elmo_surf, pickle_out)
# pickle_out.close()

In [37]:
elmo_surf2 = np.concatenate(elmo_surf,axis = 0)
elmo_surf_scaled = scale(elmo_surf2)

dum_pred = model.predict(elmo_surf_scaled,batch_size=32,verbose=1)
dum_pred = softmax(dum_pred,axis=1)
surfnew_prediction = list(map(get_lable,dum_pred))

pd.DataFrame(surfnew_prediction).to_csv("elmo_surf_pred.csv",index = False)

surfnew['elmo_results'] = surfnew_prediction

surfnew.to_csv("elmo_surf_pred.csv",index = False)